In [1]:
import pandas as pd

# Load the dataset
dataset_path = r"C:\Users\NN\Desktop\MLOPS\train.csv"
df = pd.read_csv(dataset_path)

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class HousePricePredictor:
    def __init__(self, dataset_path: str):
        self.dataset = pd.read_csv(dataset_path)
        self.model = None
        self.scaler = None
        self._prepare_data()

    def _prepare_data(self):
        # Select some relevant features for simplicity
        features = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF']
        target = 'SalePrice'

        # Filter the dataset to keep only relevant columns
        self.X = self.dataset[features]
        self.y = self.dataset[target]

        # Handle missing values by filling with median (simplified approach)
        self.X.fillna(self.X.median(), inplace=True)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)

        # Scale the features
        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        
        # Initialize and train the linear regression model
        self.model = LinearRegression()
        self.model.fit(X_train_scaled, y_train)

    def predict(self, features: dict):
        # Convert the features dictionary to a DataFrame
        input_data = pd.DataFrame([features])

        # Fill missing values
        input_data.fillna(self.X.median(), inplace=True)

        # Scale the input data
        input_data_scaled = self.scaler.transform(input_data)

        # Predict the house price
        return self.model.predict(input_data_scaled)[0]


In [3]:
from fastapi import FastAPI
from pydantic import BaseModel

# Define the FastAPI app
app = FastAPI()

# Define the input data model using Pydantic
class HouseFeatures(BaseModel):
    OverallQual: int
    GrLivArea: float
    GarageCars: int
    GarageArea: float
    TotalBsmtSF: float

# Mock predictor for demonstration
class HousePricePredictor:
    def predict(self, features: dict):
        return 100000 + 50000 * features["OverallQual"] + 30 * features["GrLivArea"]

# Instantiate the predictor
predictor = HousePricePredictor()

# Define a POST endpoint for predictions
@app.post("/predict")
def predict_price(features: HouseFeatures):
    prediction = predictor.predict(features.dict())
    return {"predicted_price": prediction}


In [4]:
import nest_asyncio
import uvicorn

# Apply nest_asyncio to allow uvicorn to run in the notebook
nest_asyncio.apply()

# Start the FastAPI server
uvicorn.run(app, host="127.0.0.1", port=8001)



INFO:     Started server process [2160]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:51367 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51367 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51373 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51380 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51383 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51383 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51399 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2160]
